In [5]:
from numpy import nan
import pandas as pd
import numpy as np
import os

In [6]:
#Test Classification is the prototype of the function that each team must develop to classify new data
#This function must handle all the operation to: read in a streaming order the input file, make the earlier possibile classification, return the required data
#Input: 
# - Folder Name: The name of the folder where the experiment file is stored
# - Experiment: The experiment name that must be read 
#Output:
# - Predicted Label: the label predicted by the classifier
# - Time for classification: how much time of the input data was required to perform the classification task
# - Ranking: The Features ranked according to the team solution

def TestClassification(FolderName,Experiment):

        
    Label = ""
    Time = -1
    Ranking = []

    
    #Output example to run the script
    Label=0
    Time = 10
    Ranking=["S4","S2","S1","S3"]

    return Label, Time, Ranking

In [8]:
#The Record Performance function store the information of the performance achieved in the 1st run of the classification
def RecordPerformance(Experiment, Label, Time, Ranking):

    if not os.path.exists('First'):
        os.makedirs('First')
    
    PerformanceOutput = open("First/%s.csv"%Experiment,"w")
    PerformanceOutput.write("Experiment;Label;Time;Ranking\n")
    PerformanceOutput.write(Experiment+";"+str(Label)+";"+str(Time)+";"+str(Ranking)+"\n")
    PerformanceOutput.close()
    
    return

#The CutExperiment function is used to cut the input experiment in the time order. For each experiment, the cut is performed accoring to the time to classification declared by the team 
def CutExperiment(FolderName, Experiment, Time):

    if not os.path.exists('Cut'):
        os.makedirs('Cut')
    
    data = pd.read_csv(FolderName+"/%s.csv"%Experiment,sep=",")  

    df = pd.DataFrame(columns = ["c1","c2"])
    
    for i in range(len(data)):
        field = data.iloc[i][0]
        records = eval(data.iloc[i][1])[:Time]

        df = df.append({"c1": field, "c2": records}, ignore_index=True)    
    
    columns =list(data.columns)
    df.columns = ["",columns[1]]
    df.to_csv("Cut/%s.csv"%Experiment,index=False)
    return

#ComparePerfomance check if the team achieved the same performance in the 1st and in the 2nd run 
def ComparePerformance(Experiment,Label, Time, Ranking):
    
    Performance = pd.read_csv("First/%s.csv"%Experiment,sep=";")        
        
    if(Performance["Label"].iloc[0]!=Label): return False
    if(Performance["Time"].iloc[0]!=Time): return False
    if(Performance["Ranking"].iloc[0]!=str(Ranking)): return False
        
    return True

#GetWorst function returns the worst performance in case the 1st and 2nd run performance does not match
def GetWorst(Experiment, Label, Time, Ranking):
    
    Performance = pd.read_csv("First/%s.csv"%Experiment,sep=";")  
    
    if(Performance["Label"].iloc[0]!=Label): return Performance["Label"].iloc[0], -1, Performance["Ranking"].iloc[0]
    if(Performance["Time"].iloc[0]!=Time): return Performance["Label"].iloc[0], Performance["Time"].iloc[0], Performance["Ranking"].iloc[0]
    if(Performance["Ranking"].iloc[0]!=str(Ranking)): return Performance["Label"].iloc[0], Performance["Time"].iloc[0], Performance["Ranking"].iloc[0]
    
    return

#Logperformance function stores the final performance. Only this performance will be used to compute the Penalty score of each team
def LogPerformance(Experiment,Label, Time, Ranking):

    if not os.path.exists('Results'):
        os.makedirs('Results')
        
    PerformanceOutput = open("Results/%s.csv"%Experiment,"w")
    PerformanceOutput.write("Experiment;Label;Time;Ranking\n")
    PerformanceOutput.write(Experiment+";"+str(Label)+";"+str(Time)+";"+str(Ranking)+"\n")
    PerformanceOutput.close()
    
    return

In [9]:
#Example of the validation pipleline by using a single experiment.
#Data/ is the folder where the experiment is stored
#class_0_0_data is the experiment name
#Cut/ is the folder where only the cut experiment will be saved

def main():
    
    FolderName = "Data/"
    Experiment = "class_0_0_data"
    Label, Time, Ranking = TestClassification(FolderName,Experiment)
    
    RecordPerformance(Experiment, Label, Time, Ranking)
    CutExperiment(FolderName,Experiment,Time)
    
    FolderName = "Cut/"
    Label, Time, Ranking = TestClassification(FolderName,Experiment)
    
    Equal = ComparePerformance(Experiment,Label, Time, Ranking)
    if(Equal==False):
        Label, Time, Ranking = GetWorst(Experiment,Label, Time, Ranking)
    
    LogPerformance(Experiment,Label, Time, Ranking)
    return

main()